In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

In [49]:
user='G'

if user=='S':
    mandat_dep=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
    df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

if user=='G':
    mandat_dep=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandatdef.csv") 
    df_depu=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/listeDepute_complete.csv")

<h1> I. Base mandat

In [50]:
debut_mandat=[]
for dates in mandat_dep["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
mandat_dep["Debut Mandat"]=np.array(debut_mandat)
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


In [51]:
df_depu.head(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort
0,0,Ludovic Pajot,18 novembre 1993,NaN,NaN
1,1,Typhanie Degois,6 janvier 1993,NaN,NaN
2,2,Lénaïck Adam,19 février 1992,NaN,NaN
3,3,Pierre Henriet,26 novembre 1991,NaN,NaN
4,4,Robin Reda,10 mai 1991,NaN,NaN


In [55]:
mandat_dep.head(5)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007
2,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2012
3,0,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,Christine Cloarec-Le Nabour,le 21 juin
4,0,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,Pascal Terrasse,1er juin 1997


In [79]:
i=mandat_dep.groupby("Nom").count()[["Département","Régime politique"]]
i[i["Département"]>2][i["Régime politique"]>2]

<ipython-input-79-70394de0bc12>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  i[i["Département"]>2][i["Régime politique"]>2]


,Département,Régime politique
Nom,,
Abderrahmane Bentounès,3,3
"Abdon, Patrocle, Frédéric Hély d'Oissel",3,3
Abel Ferry,3,3
Abel Lefèvre,4,4
Abel Poulain,3,3
...,...,...
Yvon Bourges,5,5
"Yvon, Pierre, Stanislas Delbos",8,8
Éric Straumann,3,3


In [53]:
df_depu[["Nom","Né le"]].head(2)
mandat_dep[["Nom","Debut Mandat"]].head(2)
age_dep=mandat_dep.set_index('Nom').join(df_depu[["Nom","Né le"]].set_index('Nom'), on='Nom')
mandat_dep.reset_index()
mois={'janvier':1,'février':2, 'mars':3, 'avril':4, 'mai':5, 'juin':6, 'juillet':7, 'août':8, 'septembre':9, 'octobre':10, 'novembre':11, 'décembre':12}
def transtype_date(x):
    x=str(x)        
    date_texte=x.split() 
    if len(date_texte)>1 :
        if  date_texte[1] in (mois.keys()):
            if date_texte[0]=="1er":
                date_texte[0]='1'
            date_texte[1]=str(mois[date_texte[1]])
            date_nombre='/'.join(date_texte)
            return pd.to_datetime(date_nombre)
    else:
        return pd.NaT


age_dep["Date de naissance"]=age_dep["Né le"].apply(lambda x: transtype_date(x))
age_dep["Date debut mandat"]=age_dep["Debut Mandat"].apply(lambda x: transtype_date(x))
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat
Nom,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20


In [38]:
age_dep['age depute']=age_dep['Date debut mandat']-age_dep['Date de naissance']#on crée la variable age au debut du mandat, en jour
age_dep['age en année']=age_dep['age depute'].apply(lambda x: float(x.days / 365.25))#et on la transforme en années
#age_dep=age_dep[['Département','Groupe', 'Législature', 'Mandat', 'Régime politique', 'age en année','Date debut mandat']]

In [39]:
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,age en année
Nom,,,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19,13734 days,37.601643
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20,15561 days,42.603696


In [46]:
age_dep[age_dep["age en année"]<0]

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,age en année
Nom,,,,,,,,,,,,
Paul Giacobbi,0,Corse,Radical et radical-socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,4 juin 1957,1957-04-06,1945-10-21,-4185 days,-11.457906
Paul Giacobbi,0,Corse,Radical et radical-socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,4 juin 1957,1957-04-06,1945-10-21,-4185 days,-11.457906
Paul Giacobbi,0,Corse,Radical et radical-socialiste,2e Assemblée nationale constituante,Du 2 juin 1946 au 27 novembre 1946,Gouvernement provisoire de la République franç...,2 juin 1946,4 juin 1957,1957-04-06,1946-02-06,-4077 days,-11.162218
Paul Giacobbi,0,Corse,Radical et radical-socialiste,2e Assemblée nationale constituante,Du 2 juin 1946 au 27 novembre 1946,Gouvernement provisoire de la République franç...,2 juin 1946,4 juin 1957,1957-04-06,1946-02-06,-4077 days,-11.162218
Paul Giacobbi,0,Corse,Républicain radical et radical-socialiste,Ire législature,Du 10 novembre 1946 au 5 avril 1951,Quatrième République - Assemblée nationale,10 novembre 1946,4 juin 1957,1957-04-06,1946-10-11,-3830 days,-10.485969
...,...,...,...,...,...,...,...,...,...,...,...,...
Jean Rousseau,0,Seine,Modérés,Convention nationale,Du 27 février 1795 au 26 octobre 1795,Révolution,27 février 1795,21 juin 1943,1943-06-21,1795-02-27,-54169 days,-148.306639
Pierre André,0,Bas-Rhin,Gauche,Conseil des Cinq-Cents,Du 13 avril 1798 au 10 novembre 1799,Révolution,13 avril 1798,27 octobre 1903,1903-10-27,1798-04-13,-38546 days,-105.533196
Pierre André,0,Bas-Rhin,Gauche,Conseil des Cinq-Cents,Du 13 avril 1798 au 10 novembre 1799,Révolution,13 avril 1798,27 octobre 1903,1903-10-27,1798-04-13,-38546 days,-105.533196


<h1> II.Table député

<h3> Nombre de mandat par député

In [28]:
nb_mandat=age_dep.groupby("Nom").count()
nb_mandat=pd.DataFrame(nb_mandat).rename(columns={"age en année":"nombre de mandat"})
nb_mandat.head()

,Département,Groupe,Législature,Mandat,Régime politique,nombre de mandat,Date debut mandat
Nom,,,,,,,
"(Louis, Camille, Alexandre, Marie Dubochet",1,0,1,1,1,1,1
"Aaron, Jean-François Crassous",1,1,1,1,1,1,1
Abbas Gueye,1,1,1,1,1,1,1
Abbes Moulessehoul,1,1,1,1,1,1,1
Abdallah Tebib,1,1,1,1,1,1,1


In [9]:
df_depu_b=df_depu.join(nb_mandat,on="Nom",lsuffix=" ")
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat
1421,1421,François Huwart,20 juin 1947,Inconnu,Inconnu,1.0
9941,9941,"Charles, Eléonore, François Plan de Siéyès",8 novembre 1812,Drôme,Inconnu,2.0
14795,14795,"Jean, Adrien Bigonnet",15 mars 1755,Inconnu,Inconnu,2.0
3415,3415,Modibo Keita,4 juin 1915,Inconnu,Inconnu,2.0
5673,5673,Edouard Jonas,9 mai 1883,Seine,Paris,1.0


In [10]:
premier_mandat=age_dep.groupby("Nom")["age en année"].min()
premier_mandat.head()

Nom
(Louis, Camille, Alexandre, Marie Dubochet    45.689254
Aaron, Jean-François Crassous                 49.267625
Abbas Gueye                                   37.470226
Abbes Moulessehoul                            38.811773
Abdallah Tebib                                60.766598
Name: age en année, dtype: float64

In [11]:
age_dep=age_dep.reset_index()

In [12]:
premier_mandat_dpt=age_dep.groupby("Nom")[["Département","Nom"]].head(1)
premier_mandat_dpt=pd.DataFrame(premier_mandat_dpt).rename(columns={"Département":"premier dpt"}).set_index("Nom")
premier_mandat_dpt.head(3)

,premier dpt
Nom,
Damien Meslot,Territoire-de-Belfort
Christine Cloarec-Le Nabour,Ille-et-Vilaine
Pascal Terrasse,Ardèche


<h3> Caractéristiques du premier mandat

In [13]:
df_depu_b=df_depu_b.join(premier_mandat,on="Nom",lsuffix="")
df_depu_b=df_depu_b.join(premier_mandat_dpt,on="Nom",lsuffix="")

In [14]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt
5921,5921,André Goirand,3 avril 1879,Seine,Paris,1.0,45.672827,Deux-Sèvres
6022,6022,Antoine-Marius Gallet,15 août 1877,Ain,Ain,1.0,58.551677,Ain
7669,7669,"Charles, Marie, Joseph Borne",1er février 1850,Doubs,Doubs,2.0,48.588638,Doubs
12484,12484,"Alexandre, Jean, Louis Bourdon du Rocher",29 juin 1777,Sarthe,Lieu,2.0,50.381930,Sarthe
12622,12622,"Louis, Victor de Caux de Blaquetot",23 mai 1775,Nord,Yvelines,2.0,52.484600,Nord


In [15]:
df_depu_b["élection locale"]=(" "+df_depu_b["Département de naissance"])==df_depu_b["premier dpt"]

In [16]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt,élection locale
16263,16263,"Louis, Bonaventure Vanhoenacker",16 janvier 1734,Nord,Nord,1.0,56.980151,Nord,True
2282,2282,Gérard César,19 décembre 1934,Gironde,Inconnu,3.0,41.776865,Gironde,True
8224,8224,Félix-François Deville,14 mars 1841,Aisne,Aisne,2.0,48.240931,Aisne,True
6223,6223,"Isidore, François Tournan",31 août 1874,Gers,Deux-Sèvres,2.0,35.926078,Gers,True
3616,3616,Jean Capdeville,8 novembre 1912,Gers,Loire-Atlantique,4.0,33.193703,Seine-Inférieure,False


<h1> III. Base département

In [17]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements
dep_possible.remove(" Département d'élection inconnu")
dep_possible.remove(np.nan)

In [18]:
dep_possible[:3]

[' Territoire-de-Belfort', ' Ille-et-Vilaine', ' Ardèche']

In [19]:
def pourcentageTerroir(basedep) :#une fonction qui donne le pourcentage de mandats occupés par des députés nés dans leur département d'élection
    basedep_1=basedep[basedep['Département']==' '+basedep['Département de naissance']]
    nbreterroir=basedep_1[basedep_1['Département de naissance']!='Inconnu']['Département'].count()
    i=basedep[basedep['Département de naissance']!='Inconnu']['Département'].count()
    if i!=0:
        pourcTerroir=nbreterroir/i
    else:
        return np.nan
    return pourcTerroir

In [20]:
df=age_dep.reset_index()

In [21]:
dept_naiss=age_dep.set_index('Nom').join(df_depu[["Nom","Né le", "Département de naissance"]].set_index('Nom'), on='Nom')
dept_naiss=dept_naiss.reset_index()
dept_naiss.head()

,Nom,Département,Groupe,Législature,Mandat,Régime politique,age en année,Date debut mandat,Né le,Département de naissance
0,Damien Meslot,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,37.601643,2002-06-19,11 novembre 1964,Inconnu
1,Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,42.603696,2007-06-20,11 novembre 1964,Inconnu
2,Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,47.605749,2012-06-20,11 novembre 1964,Inconnu
3,Christine Cloarec-Le Nabour,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,NaN,NaT,28 octobre 1964,Inconnu
4,Pascal Terrasse,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,32.197125,1997-01-06,26 octobre 1964,Gard


In [22]:

w,x,y,z,u=[],[],[],[],[]
for dep in dep_possible :
    w+=[df[df['Département']==dep]['age en année'].mean()]#on prend la moyenne d'age dans le département
    x+=[len(df[df['Département']==dep])]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['Nom'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    u+=[pourcentageTerroir(dept_naiss[dept_naiss["Département"]==dep])]
    
    
carac_dep_bis=np.array([w, x, y, z,u])#on crée un dataframe avec les informations
table_depmt = pd.DataFrame(carac_dep_bis, index=['age moyen des deputes', 'nombre_mandat', 'nombre député','nombre moyen de mandats par député', 'locaux'],columns=dep_possible)
table_depmt.T.sample(10)

,age moyen des deputes,nombre_mandat,nombre député,nombre moyen de mandats par député,locaux
Vendée,49.478806,351.0,147.0,2.387755,0.711765
Yonne,48.862315,357.0,147.0,2.428571,0.505519
Départements de l'Empire français aujourd'hui en Italie,47.717454,89.0,89.0,1.000000,0.000000
Alpes-de-Haute-Provence,55.715872,9.0,7.0,1.285714,0.000000
Territoire français des Affars et des Issas,39.978097,4.0,3.0,1.333333,0.000000
Bas-Rhin,47.690881,424.0,197.0,2.152284,0.750000
Soudan-Niger,43.079398,4.0,3.0,1.333333,0.000000
Comores,51.297833,15.0,4.0,3.750000,0.000000
Tarn,47.211667,329.0,149.0,2.208054,0.662921
Hauts-de-Seine,54.076791,184.0,67.0,2.746269,0.000000


In [25]:
df_depu_b.sample(100)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt,élection locale
12964,12964,"Raymond, Jacques, Pascal Rivals-Gincla",16 avril 1770,Aude,Aude,1.0,45.078713,Aude,True
15369,15369,Justin Girod-Chantrans,26 septembre 1750,Doubs,Doubs,1.0,51.496235,Doubs,True
13722,13722,"Pierre, François, Joseph Robert",21 janvier 1763,Inconnu,Inconnu,1.0,29.883641,Seine,False
1344,1344,Alain Le Vern,8 mai 1948,Finistère,Inconnu,3.0,39.854894,Seine-Maritime,False
3635,3635,Louis Dupont,7 septembre 1912,Meurthe,Alpes-Maritimes,12.0,-96.673511,Meurthe-et-Moselle,False
...,...,...,...,...,...,...,...,...,...
975,975,Geneviève Gosselin-Fleury,22 février 1954,Manche,Inconnu,NaN,NaN,NaN,False
9917,9917,"Adolphe, Elisabeth, Joseph, Octavien Pourroy d...",19 mars 1813,Isère,Paris,1.0,58.370979,Isère,True
2892,2892,Jacques Huyghues des Étages,15 novembre 1923,Seine,Inconnu,4.0,49.223819,Nièvre,False
6686,6686,Jean-Baptiste Giray,13 octobre 1866,Isère,Isère,1.0,47.975359,Isère,True


In [81]:
table_depute=df_depu_b
table_depute.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_depute.csv")

In [30]:
table_depmt.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_dpmt.csv")